In [1]:
#imports
import numpy as np
import math

In [2]:
# Load Data
data = np.genfromtxt('scale-d-10d.csv', delimiter=' ')
data = data[:,0:9]

labels = np.genfromtxt('scale-d-10d.csv', delimiter=' ',dtype="|U5")
labels = labels[:,10]

In [4]:
def proj_true_p_signature(M):
    # n = number of data objects
    n = M.shape[0]
    # number of bins
    bins = int(1 + math.log(n,2))
    # For every bin in every attribute, its support is computed
    # SupportSet = {x ∈ D | x.aj ∈ S }
    supp_set = np.empty(n)
    for i in range(data.shape[1]):
        x = 2


    return None

In [5]:
print(data)
print(data.shape)

print(labels)
print(labels.shape)


print(int(1 + math.log(data.shape[0],2)))

[[ 0.65264191 -0.00371625  0.01521223 ...  0.21708319  0.78426444
   0.27407853]
 [ 1.01930291  0.35519293  0.47796839 ...  0.60008908  0.35612478
   0.24724175]
 [ 0.5940366  -0.33190342 -0.10154468 ...  0.68686449  0.53546747
   1.27822433]
 ...
 [ 0.84569877  0.35925799  0.64490759 ...  0.87088832  0.63797899
   0.87146842]
 [ 0.27499317  0.49355767  0.20943054 ...  0.93080848  0.72043327
   1.42526599]
 [ 0.39640662  0.57944655  0.980204   ...  0.0876678   0.15534795
   0.67756318]]
(10000, 9)
['C1' 'C1' 'C1' ... 'C100' 'C100' 'C100']
(10000,)
14


In [6]:
M = np.array([[1, 2], [4, 5], [3, 3], [6, 7]])
supp_set = np.empty(int(1 + math.log(M.shape[0],2)),dtype=object)
temp1 = np.array([])
temp2 = np.array([])
temp3 = np.array([])
for i in range(M.shape[1]):
    for j in range(M.shape[0]):
        if(M[i,j]>=3)
            temp1 = np.append(temp1, M[i,j])
        if(M[i,j]>=6 and M[i,j]<3)
            temp2 = np.append(tem2, M[i,j])
        if(M[i,j]>6)
            temp3 = np.append(tem3, M[i,j])
    
    supp_set[0] = temp1
    supp_set[1] = temp2
    supp_set[3] = temp3
    
supp_set
    

SyntaxError: invalid syntax (<ipython-input-6-589360ffa9f2>, line 8)